# Compute BM25 baseline and view NDCG

This notebook just does a basic analysis of poor performing queries with a BM25 baseline against the WANDS (Wayfair Annotated Dataset) search query dataset


## Boilerplate

Install deps, mount GDrive, prompt for your OpenAI Key (placed in your GDrive)

In [ ]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)

  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-lpq5w74u
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-lpq5w74u
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 38a087b480422fb5f29fea8b25fbfb25f3492da3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Introducing our baseline search strategy

Here is a very basic search strategy that executes a naive BM25 search. Notice

* In the constructor we initialize some weights (name and body weights)
* In `search` we search each field, summing scores as we go

In [ ]:
from cheat_at_search.tokenizers import snowball_tokenizer

snowball_tokenizer("red couches for two forts")

['red', 'couch', 'for', 'two', 'fort']

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer
import numpy as np
from cheat_at_search.strategy.strategy import SearchStrategy


class BM25Search(SearchStrategy):
    def __init__(self, products,
                 name_boost=9.3,
                 description_boost=4.1):
        super().__init__(products)
        self.index = products
        # ***
        # Remember the boosts for later
        self.name_boost = name_boost
        self.description_boost = description_boost

        # ***
        # Index product name and product description (via snowball)
        self.index['product_name_snowball'] = SearchArray.index(
            products['product_name'], snowball_tokenizer)
        self.index['product_description_snowball'] = SearchArray.index(
            products['product_description'], snowball_tokenizer)

    def search(self, query, k=10):
        """Dumb baseline lexical search"""
        # ***
        # Tokenize the query
        tokenized = snowball_tokenizer(query)
        print(tokenized)

        # ***
        # Initialize scores to 0 (an array with a float for eac doc)
        bm25_scores = np.zeros(len(self.index))

        # ***
        # Search one token at a time, in each field, and add each docs BM25
        # score to the bm25_scores
        for token in tokenized:
            print(token)
            bm25_scores += self.index['product_name_snowball'].array.score(token) * self.name_boost
            bm25_scores += self.index['product_description_snowball'].array.score(
                token) * self.description_boost

        # ***
        # Return top k row indices
        top_k = np.argsort(-bm25_scores)[:k]
        scores = bm25_scores[top_k]
        return top_k, scores

In [ ]:
strategy = BM25Search(products)

2025-10-08 17:01:07,272 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-10-08 17:01:07,287 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-10-08 17:01:07,290 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

INFO:searcharray.indexing:Batch tokenization complete


2025-10-08 17:01:16,178 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-10-08 17:01:16,548 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


In [ ]:
top_k, scores = strategy.search("desk for kids")
top_k

['desk', 'for', 'kid']
desk
for
kid


array([21853, 17416, 18018, 41388, 23916, 21286, 25014, 38462,  9617,
        9612])

In [ ]:
top_k_results = products.iloc[top_k]
top_k_results['score'] = scores
top_k_results[['product_name', 'product_description']]

/tmp/ipython-input-565134688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_k_results['score'] = scores


,product_name,product_description
21853,kids desk chair,"pint-sized to fit at a kid 's table or desk , ..."
17416,willette kids study desk,this kids ' complete desk system is designed t...
18018,cordova kids desk,this office chair is perfect for the kids and ...
41388,kids wrting desk wood,"39.4in small desk , a good choice for you kids..."
23916,willison kids study desk,the wrought studio kid 's complete desk system...
21286,kids multifunctional desk and chair set adjust...,"kids ' adjustable height desk , kids multifunc..."
25014,monarch hill poppy kids study desk,every parent wants to create the perfect study...
38462,beseris kids desk chair,"with kids , it ’ s all about compromise . and ..."
9617,kids chair desk with storage bin,"for animated fun , grab a versatile seating op..."
9612,mickey kids study desk and chair set,this children mickey mouse chair desk is an id...


In [ ]:
from cheat_at_search.search import run_strategy
graded_bm25 = run_strategy(strategy)

['salon', 'chair']
salon
chair
['smart', 'coffe', 'tabl']
smart
coffe
tabl
['dinosaur']
dinosaur
['turquois', 'pillow']
turquois
pillow
['chair', 'and', 'a', 'half', 'reclin']
chair
and
a
half
reclin


Searching:   0%|          | 2/480 [00:00<00:28, 16.58it/s]

['sofa', 'with', 'ottoman']
sofa
with
ottoman
['acryl', 'clear', 'chair']
acryl
clear
chair
['driftwood', 'mirror']
driftwood
mirror
['home', 'sweet', 'home', 'sign']
home
sweet
home
sign
['coffe', 'tabl', 'fire', 'pit']
coffe
tabl
fire
pit
['king', 'poster', 'bed']
king
poster
bed
['ombr', 'rug']
ombr
rug
['larg', 'spoon', 'and', 'fork', 'wall', 'decor']
larg
spoon
and
fork
wall
decor
['outdoor', 'privaci', 'wall']
outdoor
privaci
wall
['bed', 'that', 'have', 'led']
bed
that
have
led
['black', '5', 'drawer', 'dresser', 'by', 'guilford']
black
5
drawer
dresser
by
guilford
['blk', '18x18', 'seat', 'cushion']
blk
18x18
seat
cushion
['closet', 'storag', 'with', 'zipper']
closet
storag
with
zipper
['chrome', 'bathroom', '4', 'light', 'vaniti', 'light']
chrome
bathroom
4
light
vaniti
light
['gurney', 'slade', '56']
gurney
slade
56
['foutain', 'with', 'brick', 'look']
foutain
with
brick
look


Searching:   1%|          | 5/480 [00:00<00:23, 19.99it/s]

['live', 'curtain', 'pearl']
live
curtain
pearl
['light', 'and', 'navi', 'blue', 'decor', 'pillow']
light
and
navi
blue
decor
pillow
['stoneford', 'end', 'tabl', 'white', 'and', 'wood']
stoneford
end
tabl
white
and
wood
['wood', 'coffe', 'tabl', 'set', 'by', 'storag']
wood
coffe
tabl
set
by
storag
['sunflow']
sunflow
['leather', 'chair']
leather
chair
['outdoor', 'welcom', 'rug']
outdoor
welcom
rug
['rooster', 'decor']
rooster
decor
['bathroom', 'vaniti', 'knob']
bathroom
vaniti
knob
['3', '1', '2', 'inch', 'drawer', 'pull']
3
1
2
inch
drawer
pull
['burnt', 'orang', 'curtain']
burnt
orang
curtain
['dark', 'gray', 'dresser']
dark
gray
dresser
['non', 'slip', 'shower', 'floor', 'tile']
non
slip
shower


...

Searching:  79%|███████▉  | 381/480 [00:11<00:03, 32.41it/s]

['aiden', 'black', 'velvet', 'bed']
aiden
black
velvet
bed
['day', 'bed', 'indian']
day
bed
indian
['midcenturi', 'tv', 'unit']
midcenturi
tv
unit
['carpet', '5x6']
carpet
5x6
['deer', 'coat', 'hook']
deer
coat
hook
['wall', 'design', 'shelf']
wall
design
shelf
['5', 'gang', 'light', 'switch']
5
gang
light
switch
['bubbl', 'guppi', 'chair']
bubbl
guppi
chair
['tye', 'dye', 'duvet', 'cover']
tye
dye
duvet
cover
['small', 'wardrob', 'grey']
small
wardrob
grey
['luau', 'string', 'light']
luau
string
light
['larg', 'base']
larg
base
['bistro', 'set', 'patio']
bistro
set
patio
['cake', 'cutleri']
cake
cutleri
['white', 'abstract']
white
abstract
['rock', 'climb', 'wall', 'kid']
rock
climb
wall
kid
['town', 'countri', 'live', 'curtain']
town
countri
live
curtain
['glow', 'in', 'the', 'dark', 'silent', 'wall', 'clock']
glow
in
the
dark
silent
wall
clock


Searching:  80%|████████  | 385/480 [00:11<00:03, 28.85it/s]

['garag', 'sport', 'storag', 'rack']
garag
sport
storag
rack
['eleph', 'frame']
eleph
frame
['adjust', 'height', 'artist', 'stool']
adjust
height
artist
stool
['window', 'wall', 'accent']
window
wall
accent
['chair', 'pillow', 'cushion']
chair
pillow
cushion
['48', 'inch', 'bathroom', 'vaniti', 'with', 'trough', 'sink']
48
inch
bathroom
vaniti
with
trough
sink
['rattan', 'truck']
rattan
truck
['hardwood', 'bed']
hardwood
bed
['worn', 'leather', 'offic', 'chair']
worn
leather
offic
chair
['garag', 'door', 'mail', 'slot']
garag
door
mail
slot
['pictur', 'to', 'hang', 'over', 'fireplac']
pictur
to
hang
over
fireplac
['emma', 'headboard']
emma
headboard
['48', 'inch', 'kitchen', 'hood']
48
inch
kitchen
hood
['rustic', 'twig']
rustic
twig
['nespresso', 'vertuo', 'next', 'premium', 'by', 'brevill', 'with', 'aeroccino']
nespresso
vertuo
next
premium
by
brevill
with
aeroccino
['pedistol', 'sink']
pedistol
sink
['54', 'in', 'bench', 'cushion']
54
in
bench
cushion
['rack', 'glass']
rack
glass


Searching: 100%|██████████| 480/480 [00:14<00:00, 33.75it/s]


In [ ]:
graded_bm25.groupby('query')['ndcg'].mean().sort_values(ascending=False)

,ndcg
query,
window wall accent,1.0
waterfall faucet,1.0
basket planter,1.0
amarillo,1.0
anti fatigue mat,1.0
...,...
podium with locking cabinet,0.0
small loving roomtables,0.0
small ladies rocker swivel recliner,0.0


## Shortcuts (saved baseline, etc)

Helpers to compute ndcg of each query and other comparison functions

In [ ]:
from cheat_at_search.search import graded_bm25, ndcgs, ndcg_delta, vs_ideal

In [ ]:
graded_bm25[['query', 'rank', 'product_name', 'grade']]

,query,rank,product_name,grade
0,salon chair,1,hair salon chair,2.0
1,salon chair,2,barberpub salon massage chair,2.0
2,salon chair,3,mercer41 hair salon chair hydraulic styling ch...,2.0
3,salon chair,4,professional salon reclining massage chair,2.0
4,salon chair,5,beauty salon task chair,1.0
...,...,...,...,...
4795,rack glass,6,wine glass rack,0.0
4796,rack glass,7,madisen hanging wine glass rack,0.0
4797,rack glass,8,kena hanging wine glass rack,0.0
4798,rack glass,9,wall mounted wine glass rack,0.0


### View random Queries in WANDS dataset with BM25

In [ ]:
ndcgs(graded_bm25).sort_values().head(20)

,ndcg
query,
large bases,0.000000
star wars rug,0.000000
small loving roomtables,0.000000
small ladies rocker swivel recliner,0.000000
promo codes or discounts,0.000000
rug for teen room,0.000000
rack glass,0.000000
podium with locking cabinet,0.000000
dull bed with shirt head board,0.000000


### Compare BM25 vs the 'ideal' search results

In [ ]:
# podium with locking cabinet	-> item type

QUERY = "star wars rug"
bm25_vs_ideal = vs_ideal(graded_bm25)
bm25_vs_ideal[bm25_vs_ideal['query'] == QUERY]

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
1730,174,star wars rug,5204,50671,Partial,1.0,1,5204,barrett multicolor area rug,1,36859,star wars ep9 coffee mug,0.0,0.0,0.0
1731,174,star wars rug,9137,50677,Partial,1.0,2,9137,geometric purple area rug,2,22734,“ star wars - darth vader ” autographed framed...,0.0,0.0,0.0
1732,174,star wars rug,9138,50676,Partial,1.0,3,9138,delcastillo geometric purple area rug,3,3275,stormtrooper star wars standup,0.0,0.0,0.0
1733,174,star wars rug,9140,50678,Partial,1.0,4,9140,oloughlin geometric purple area rug,4,28470,star wars dark side storage accent trunk,0.0,0.0,0.0
1734,174,star wars rug,9150,50679,Partial,1.0,5,9150,rodrigue geometric purple area rug,5,617,star wars stormtrooper cardboard standup,0.0,0.0,0.0
1735,174,star wars rug,12098,50670,Partial,1.0,6,12098,green/red/gray area rug,6,627,pao star wars ( rogue one ) standup,0.0,0.0,0.0
1736,174,star wars rug,22768,50669,Partial,1.0,7,22768,corina soccer time area rug,7,34916,star wars rule the galaxy rectangular pillow c...,0.0,0.0,0.0
1737,174,star wars rug,614,25429,Irrelevant,0.0,8,614,star wars rebels garazeb zeb orrelios cardboar...,8,616,star wars rebels stormtrooper cardboard standup,0.0,0.0,0.0
1738,174,star wars rug,615,25430,Irrelevant,0.0,9,615,star wars rebels kanan jarrus cardboard standup,9,21847,star wars tie fighter 10 light string lights,0.0,0.0,0.0
1739,174,star wars rug,616,25431,Irrelevant,0.0,10,616,star wars rebels stormtrooper cardboard standup,10,32602,popular characters star wars classic wall decal,0.0,0.0,0.0


In [ ]:
from cheat_at_search.wands_data import labeled_query_products

RELEVANT = 2
MEH = 1
labeled_query_products[(labeled_query_products['query'] == QUERY) &
                       (labeled_query_products['grade'] == MEH)][['query', 'product_id', 'category hierarchy']]

,query,product_id,category hierarchy
100589,star wars rug,5204,Rugs / Area Rugs / 4' x 6' Area Rugs
100593,star wars rug,9137,Rugs / Area Rugs
100594,star wars rug,9138,Rugs / Area Rugs
100595,star wars rug,9140,Rugs / Area Rugs
100596,star wars rug,9150,Rugs / Area Rugs
100600,star wars rug,12098,Rugs / Area Rugs / 2' x 3' Area Rugs
100611,star wars rug,22768,Rugs / Area Rugs


In [ ]:
from cheat_at_search.wands_data import products

products[products['product_id'] == 6243]

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,category,sub_category,cat_subcat,product_name_snowball,product_description_snowball
6243,6243,bourbon street blues iv by robert brasher - pi...,Wall Art,Décor & Pillows / Art / All Wall Art,,framewoodtype : mdf|abstractandfractals : not ...,NaN,NaN,NaN,"[framewoodtype : mdf, abstractandfractals : no...",Décor & Pillows,Art,Décor & Pillows / Art,"Terms({'paper', 'street', 'paint', 'by', 'fram...",Terms(set())


In [ ]:
products['product_class'].value_counts().head(50)

,count
product_class,
Area Rugs,1316
Accent Chairs,1169
Beds,1112
Coffee & Cocktail Tables,1085
Floor & Wall Tile,861
Bar Stools,775
Accent Pillows,698
End Tables,607
Desks,595


In [ ]:
QUERY = "white abstract"
bm25_vs_ideal[bm25_vs_ideal['query'] == QUERY]

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
4532,465,white abstract,12092,49022,Exact,2.0,1,12092,2 piece black and white abstract fennel plants...,1,42512,devante abstract white area rug,0.0,0.0,0.0
4533,465,white abstract,12695,196676,Partial,1.0,2,12695,quickstand white portable floor recessed proje...,2,41867,burnstad abstract navy/aqua/light gray/white a...,0.0,0.0,0.0


In [ ]:
QUERY = "star wars rug"
bm25_vs_ideal[bm25_vs_ideal['query'] == QUERY]

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
1730,174,star wars rug,5204,50671,Partial,1.0,1,5204,barrett multicolor area rug,1,36859,star wars ep9 coffee mug,0.0,0.0,0.0
1731,174,star wars rug,9137,50677,Partial,1.0,2,9137,geometric purple area rug,2,22734,“ star wars - darth vader ” autographed framed...,0.0,0.0,0.0
1732,174,star wars rug,9138,50676,Partial,1.0,3,9138,delcastillo geometric purple area rug,3,3275,stormtrooper star wars standup,0.0,0.0,0.0
1733,174,star wars rug,9140,50678,Partial,1.0,4,9140,oloughlin geometric purple area rug,4,28470,star wars dark side storage accent trunk,0.0,0.0,0.0
1734,174,star wars rug,9150,50679,Partial,1.0,5,9150,rodrigue geometric purple area rug,5,617,star wars stormtrooper cardboard standup,0.0,0.0,0.0
1735,174,star wars rug,12098,50670,Partial,1.0,6,12098,green/red/gray area rug,6,627,pao star wars ( rogue one ) standup,0.0,0.0,0.0
1736,174,star wars rug,22768,50669,Partial,1.0,7,22768,corina soccer time area rug,7,34916,star wars rule the galaxy rectangular pillow c...,0.0,0.0,0.0
1737,174,star wars rug,614,25429,Irrelevant,0.0,8,614,star wars rebels garazeb zeb orrelios cardboar...,8,616,star wars rebels stormtrooper cardboard standup,0.0,0.0,0.0
1738,174,star wars rug,615,25430,Irrelevant,0.0,9,615,star wars rebels kanan jarrus cardboard standup,9,21847,star wars tie fighter 10 light string lights,0.0,0.0,0.0
1739,174,star wars rug,616,25431,Irrelevant,0.0,10,616,star wars rebels stormtrooper cardboard standup,10,32602,popular characters star wars classic wall decal,0.0,0.0,0.0
